In [ ]:
# Instalar paquetes si es necesario
!pip install yfinance --quiet
!pip install matplotlib ipywidgets --quiet

# Importaciones
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

# Modelos simulados
def modelo_lstm(data):
    return data + np.random.normal(0, 1, len(data))

def modelo_arima(data):
    return data + np.cumsum(np.random.normal(0, 0.5, len(data)))

def modelo_rf(data):
    return data + np.random.normal(0, 2, len(data))

modelos = {
    'LSTM': modelo_lstm,
    'ARIMA': modelo_arima,
    'Random Forest': modelo_rf,
}

# Widgets de la interfaz
modelo_dropdown = widgets.Dropdown(
    options=list(modelos.keys()),
    value='LSTM',
    description='Modelo:',
)

fecha_inicio_picker = widgets.DatePicker(
    description='Fecha inicio:',
    value=datetime.now().date() - timedelta(days=60)
)

dias_slider = widgets.IntSlider(
    value=30,
    min=10,
    max=180,
    step=10,
    description='Días:',
)

boton_actualizar = widgets.Button(
    description='Actualizar gráfico',
    button_style='success',
    tooltip='Haz clic para actualizar el gráfico',
)

# Gráfico al pulsar el botón
def actualizar_grafico(b):
    clear_output(wait=True)
    display(ui)

    modelo_nombre = modelo_dropdown.value
    fecha_inicio = fecha_inicio_picker.value
    dias = dias_slider.value
    fecha_fin = fecha_inicio + timedelta(days=dias)

    # Descargar datos reales simulados (usamos AAPL como ejemplo)
    df = yf.download('AAPL', start=fecha_inicio, end=fecha_fin)
    if df.empty:
        print("No hay datos disponibles para ese rango.")
        return

    reales = df['Close'].values
    fechas = df.index

    # Aplicar modelo
    modelo = modelos[modelo_nombre]
    predicho = modelo(reales)

    # Mostrar gráfico (evitando duplicados)
    plt.close('all')  # Cierra cualquier figura abierta
    plt.figure(figsize=(12, 5))
    plt.plot(fechas, reales, label='Real', linewidth=2)
    plt.plot(fechas, predicho, label=f'Predicción ({modelo_nombre})', linestyle='--')
    plt.title(f'Comparativa del modelo {modelo_nombre}')
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Conectar botón
boton_actualizar.on_click(actualizar_grafico)

# Mostrar UI
ui = widgets.VBox([modelo_dropdown, fecha_inicio_picker, dias_slider, boton_actualizar])
display(ui)
